In [ ]:
import qkit
from qkit.analysis.numerical_derivative import SavgolNumericalDerivative
from qkit.measure.unified_measurements import Experiment, Axis
import qkit.measure.samples_class as sc
from qkit.measure.transport_measurement import TransportMeasurement, MeasureModes

# Probe Station Measurement Notebook
This Notebook serves to make measurements using the fancy new probe station even more easy!
You should be able to measure 100 Samples in 5 Minutes on your own, while the data is recorded automatically.

Cells requiring your input are preceded by **bold text indicating user information.**

First, Qkit and the device is initialized. **Input a unique run_id and your name below.**

In [ ]:

qkit.cfg['load_visa']=True
qkit.cfg['run_id']='Probestation_4' # Increment to current run ID.
qkit.cfg['user']=r'Urs' # Put your name here
qkit.start()

Load the multimeter used for measurement. Currently, we are using a Keysight B2900 Desktop Multimeter. If no one messed with the cables, everything should be configured correctly.

In [ ]:
# Connect to Desktop Multimeter
IVD = qkit.instruments.create_from_config("probestation-multimeter", reset=True)

## Configure the Multimeter
We need the multimeter to do the following:
- Use 4-Wire Resistance Measurements
- Calculate Resistance
- Display only Channel 1

In [ ]:
IVD.set_defaults(1) # Configures 4-Wire IV-Measurements on channel 1. Note: 1 is preset 1, not channel 1.
IVD.set_display('single1')

- Set Bias Current to user defined value
- Limit Voltage accross DUT

In [ ]:
SENSE_LIMIT = 1 # V, prevents blown junctions due to overvoltage
IVD.set_sense_limit(SENSE_LIMIT) # Configure compliance limit in Volts
IVD.set_sense_nplc(1)
IVD.set_bias_delay(15e-6)
IVD.set_sense_delay(15e-6)

## Labeling the Sample
We provide some Information about the sample. **The sample name (usually the chip number and the chiplet) and a comment are needed.**

In [ ]:
sample = sc.Sample()
sample.name = 'US16-23' # Change to something unique
sample.comment = 'Hope for the Best.' # Optional comment here
print(sample.get_all())

## Configure the Transport Measurement
A room temperature IV curve is recorded. In this case, more data is better, allowing for automatic detection of defective samples (short circuit and open line). A 4 quadrant sweep is used.

**Update the maximum current in order not to destroy your sample!**

In [ ]:
tr = TransportMeasurement(iv_device=IVD, mode=MeasureModes.IV)
tr.with_analysis(SavgolNumericalDerivative())

CURRENT = 50e-6
tr.add_sweep_4quadrants(0, CURRENT, step=CURRENT/20, offset=0)
#tr.add_sweep_halfswing(amplitude=CURRENT, step=CURRENT/10, offset=0)

We introduce a little hack to efficiently use the transport measurement class to capture multiple samples. We present ourselves as a 3D-Measurement. The outer two dimensions are the x- and y-Position on our chip, for which the z (current bias) sweep is performed.

**Update the labels to match your sample!**

x_values is the outer loop and goes from 1 to 10 (both inclusive).
y_values is the inner loop and goes from A to J (both inclusive).
Hence the measurement sequence is 1A, 2A, ..., 10A, 1B, 2B, ..., 10J.

Labels can be changed here:

In [ ]:
x_values = list(range(2, 10 + 1))
e = Experiment(name='rt_transport', sample=sample)
with e.sweep(lambda x: input(f"Confirm {x} [ENTER]:"), axis=Axis(name='SampleIndex', range=x_values)) as x_sweep:
    x_sweep.measure(tr)
e.run()